# Black-Scholes-Merton Model

This notebook explores the Black-Scholes model, from parameter estimation to simulating stock prices and understanding implied volatility.
We'll cover key concepts like historical volatility, geometric Brownian motion, and volatility surfaces, building towards pricing European options and discussing volatility dynamics.

## Table of Content

1. [Introduction](#1-introduction)
2. [Black-Scholes Formula](#2-black-scholes-formula)
3. [Parameters Estimation](#3-parameters-estimation)
    1. [Historical Volatility](#31-historical-volatility)
    2. [Rough Volatility Aspect](#32-rough-volatility-aspect)
4. [Simulating Geometric Brownian Motion (GBM)](#4-simulating-geometric-brownian-motion-gbm)
5. [Implied Volatility](#5-implied-volatility)
6. [Stock returns are normally distributed ?](#6-stock-returns-are-normally-distributed-)

## 1. Introduction <a id="1-introduction"></a>

The Black-Scholes model is one of the most widely-used methods for pricing European-style options.
It was developed by Fischer Black, Myron Scholes, and Robert Merton in 1973.

We'll cover:
- Estimating key parameters like volatility
- Simulating asset prices using GBM
- Understanding implied volatility and related concepts like the volatility smile and surface

#### Stock Price Dynamics

The stock price in the Black-Scholes model follows a **Geometric Brownian Motion (GBM)** and it is defined by the **Stochastic Differential Equation (SDE)**:

$$
    dS_t = \mu S_t dt + \sigma S_t dW_t
$$

where $S_t$ is the asset price at time $t$, $\mu$ is its drift, $\sigma$ is its volatility, and $W_t$ is a Brownian motion.

The introduction of the equation for $S_t$ comes from the fact that, intuitively, $dS_t/S_t$ is the return on the spot, and that it is approximately constant, equal to $\mu dt$, with a (small) random perturbation $\sigma dW_t$.
The *amplitude* of this perturbation is measured by $\sigma$, a very important parameter known as the volatility of the asset $S_t$.

When using **Ito’s Lemma**, the solution for the Black-Scholes SDE is:

$$
    S_t = S_0 e^{(\mu-\frac{\sigma^2}{2})t + \sigma W_t}
$$

## 2. Black-Scholes Formula <a id="2-black-scholes-formula"></a>

The Black-Scholes formula for pricing a European Call Option is:
$$
C = S_0 N(d_1) - K e^{-rT} N(d_2),
$$
and the formula for pricing a European Put Option is:
$$
P = -S_0 N(-d_1) + K e^{-rT} N(-d_2),
$$

Where:
- $S_0$ is the current stock price
- $K$ is the strike price
- $T$ is the time to expiration (in years)
- $r$ is the risk-free interest rate
- $N$ is the cumulative distribution function of the standard normal distribution
- $d_1$ and $d_2$ are defined as:
$$
    d_1 = \frac{\log(S_0 / K) + T(r + \sigma^2 / 2)}{\sigma \sqrt{T}}
$$
$$
    d_2 = d_1 - \sigma \sqrt{T}
$$

In [61]:
# Importing necessary libraries for option pricing and data analysis
import datetime as dt

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = "plotly"

In [62]:
def black_scholes(S0, K, T, r, sigma, payoff="call"):
    """
    Calculates the Black-Scholes price for European Call or Put options.

    Parameters
    ----------
    S0 :
        The current price of the underlying asset (e.g., stock).
    K :
        The strike price of the option.
    T :
        The time to expiration in years (e.g., 1 month = 1/12).
    r :
        The risk-free interest rate (e.g., 0.05 for 5%).
    sigma :
        The volatility of the underlying asset (e.g., 0.2 for 20%).
    payoff : str
        The type of option, either "call" or "put". Default is "call".

    Returns
    -------
        The Black-Scholes price of the option (either call or put, depending on the `payoff` parameter).

    Raises
    ------
    ValueError:
        If the `payoff` is not "call" or "put".
    """
    d1 = (np.log(S0 / K) + T * (r + 0.5 * sigma**2)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if payoff.lower() == "call":
        return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif payoff.lower() == "put":
        return -S0 * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(-d2)

    raise ValueError("payoff must be either 'call' or 'put'")

In [63]:
# Fetching the market data using Yahoo Finance for the AAPL ticker

SYMBOL: str = "AAPL"

ticker = yf.Ticker(SYMBOL)
stock_data = ticker.history(period="5y")
closes = stock_data["Close"]
closes.head()

Date
2019-09-16 00:00:00-04:00    53.196217
2019-09-17 00:00:00-04:00    53.389751
2019-09-18 00:00:00-04:00    53.890518
2019-09-19 00:00:00-04:00    53.452663
2019-09-20 00:00:00-04:00    52.671280
Name: Close, dtype: float64

Log returns are a way to express the percentage change in the price of an asset.
Over a small time interval $\Delta t$ it is defined as:
$$
r_t = \log(S_{t+\Delta t}/S_{t}).
$$

In [64]:
log_returns = np.log(closes / closes.shift(1)).dropna()
log_returns.head()

Date
2019-09-17 00:00:00-04:00    0.003632
2019-09-18 00:00:00-04:00    0.009336
2019-09-19 00:00:00-04:00   -0.008158
2019-09-20 00:00:00-04:00   -0.014726
2019-09-23 00:00:00-04:00    0.004537
Name: Close, dtype: float64

We observe that, under the Black-Scholes model, log-returns of the stock price are normally distributed.
The log-return over a time interval $\Delta t$ is given by:

$$
r_t = \left(\mu - \frac{\sigma^2}{2}\right)\Delta t + \sigma\left(W_{t+\Delta t} - W_t\right),
$$

Since $W_{t+\Delta t} - W_t \sim \mathcal{N}(0, \Delta t)$, we have:

$$
r_t \sim \mathcal{N}\left(\left(\mu - \frac{\sigma^2}{2}\right)\Delta t,\ \sigma^2 \Delta t\right).
$$

Given that we are using daily data, $\Delta t = \frac{1}{252}$, as there are approximately 252 trading days in a year, not 365.

## 3. Parameters Estimation

In the Black-Scholes model, volatility is one of the most critical inputs.
We can estimate volatility using historical data or implied volatility from market prices that we will discuss later.

### 3.1. Historical Volatility

**Historical volatility** is defined as the standard deviation of the stock’s log returns over a specific period.
It reflects the variability of the stock’s returns based on historical data.

Given a series of stock prices $(S_{\Delta t}, S_{2\Delta t}, \ldots, S_{n\Delta t})$, the corresponding log returns $(r_{\Delta t}, r_{2\Delta t}, \ldots, r_{(n-1)\Delta t})$ are equal to:

$$
    (f(W_{2\Delta t}-W_{\Delta t}),f(W_{3\Delta t}-W_{2\Delta t}),\ldots,f(W_{n\Delta t}-W_{(n-1)\Delta t}))
$$

We deduce that the log returns are independent and identically distributed (iid) normal random variables:

$$
    r_{k\Delta t} \sim \mathcal{N}\left(\left(\mu - \frac{\sigma^2}{2}\right)\Delta t, \sigma^2 \Delta t\right),
$$

The standard deviation of these log returns, computed over a specific historical period, provides an estimate of the historical volatility.

In [65]:
# Annualized volatility, 252 trading day in a year
sigma_hat = np.sqrt(252 * log_returns.var())
mu_hat = 252 * log_returns.mean() + (sigma_hat ** 2 / 2)

print(f"sigma_hat = {sigma_hat}\nmu_hat = {mu_hat}")

sigma_hat = 0.31661371559483403
mu_hat = 0.33011053389387246


### 3.2 Rough Volatility Aspect

To capture a more dynamic view of volatility that adjusts to recent market movements, we compute rolling volatility.
This measure, calculated as the standard deviation of log returns over a fixed window of time (e.g., 2 days), provides a time-varying estimate of volatility. The rolling volatility highlights the rough aspects of volatility by reflecting short-term fluctuations in stock prices.

Using a rolling window of 2 days, we can observe how volatility changes with recent price movements. However, this approach would be even more insightful with intra-day data and a smaller rolling window, which could reveal finer details of volatility patterns. While rough volatility is an interesting concept and offers deeper insights into market behavior, it is beyond the scope of this notebook.

Here’s how to compute and plot the 2-day rolling volatility:

In [66]:
# If we have access to intra-day data (minute-by-minute or hourly prices), that will show even more granularity
WINDOW_SIZE: int = 2

# Compute rolling volatility with a 2-day window
rolling_volatility = np.sqrt(252 * log_returns.rolling(window=WINDOW_SIZE).var())

# Plot stock price and rolling volatility
fig = make_subplots(rows=2, cols=1, subplot_titles=["Stock price", "Volatility"], shared_xaxes=True)
fig.add_trace(go.Scatter(x=closes.index, y=closes, name="Stock price"), row=1, col=1)
fig.add_trace(go.Scatter(x=rolling_volatility.index, y=rolling_volatility, name="Volatility"), row=2, col=1)
fig.update_layout(title="Stock Price and Rolling Volatility (2-day Window)", height=800, showlegend=False)
fig.show()

## 4. Simulating Geometric Brownian Motion (GBM)

To simulate GBM, we utilize the fact that increments of Brownian motion are independent and normally distributed.

In [67]:
def geometric_brownian_motion(S0: float, mu: float, sigma: float, T: float, steps: int, d: int = 1) -> np.ndarray:
    """
    Simulate a Geometric Brownian Motion (GBM) process.
    This function generates simulated paths of a Geometric Brownian Motion (GBM) for a given set of parameters.
    The GBM model is often used to simulate stock prices and other financial time series.

    Parameters
    ----------
    S0 : float
        The starting value of the simulated stock price.
    mu : float
        Drift coefficient.
    sigma : float
        Volatility coefficient.
    T : float
        Total time of the simulation, expressed in years. This is the length of the simulation period.
    steps : int
        Number of discrete time steps in the simulation.
    d : int (optional) :
        Number of dimensions (d). Default is 1. If d > 1, the function generates multiple paths of GBM.

    Returns
    -------
    np.ndarray
        An array of simulated GBM paths. If d = 1, returns a 1-dimensional array of simulated prices.
        If d > 1, returns a 2-dimensional array with each row representing a different path.
    """

    deltatime = T / steps
    t = np.linspace(0.0, T, steps)
    Wt = np.cumsum(norm.rvs(scale=np.sqrt(deltatime), size=(d, steps - 1)), axis=1)
    St = S0 * np.hstack((np.ones((d, 1)), np.exp((mu - sigma ** 2 / 2) * t[1:] + sigma * Wt)))
    return St.flatten() if d == 1 else St

Simulating a Geometric Brownian Motion (GBM) path using the estimated parameters from our historical data and compare it to the actual stock price.

In [79]:
S0 = closes.iloc[0]
n = len(closes)
T = n / 365.0

# Simulate GBM path
gbm = geometric_brownian_motion(S0, mu_hat - (sigma_hat ** 2 / 2), sigma_hat, T, n)

# Plot stock price and GBM simulation
fig = go.Figure()
fig.add_trace(go.Scatter(x=closes.index, y=closes, name="Stock Price"))
fig.add_trace(go.Scatter(x=closes.index, y=gbm, name="GBM"))
fig.update_layout(title="Stock Price vs GBM Simulation", height=600)
fig.show()

## 5. Implied volatility

#### Implied Volatility vs. Historical Volatility

In financial markets, **Implied Volatility (IV)** and **Historical Volatility (HV)** are two essential measures of risk, each offering different insights.

**Implied Volatility** is derived from the market prices of options and reflects the market’s expectations of future volatility. It represents the level of volatility that, when used in an options pricing model like Black-Scholes, equates the theoretical price of the option with its market price. IV is forward-looking and adjusts in real-time based on market sentiment and current conditions. However, it relies on the assumptions of the pricing model and can be influenced by market dynamics such as liquidity.

In contrast, **Historical Volatility** measures the variability of an asset's price over a past period. It is calculated as the standard deviation of historical returns, providing a measure of how much the asset's price has fluctuated historically. While it offers a stable and objective view of past price behavior, HV does not predict future volatility or reflect current market conditions.

In essence, IV provides insights into future expectations and is crucial for options trading, while HV offers a perspective on past risk and price fluctuations, useful for historical analysis and risk management.

In [80]:
from scipy.optimize import fsolve

def compute_implied_volatility(market_price, S0, K, T, r, sigma0, payoff):
    """
    Compute the implied volatility of an option given its market price.

    Parameters
    ----------
    market_price :
        The current market price of the option.
    S0 :
        The current price of the underlying asset (stock price).
    K :
        The strike price of the option.
    T :
        The time to maturity of the option, expressed in years.
    r :
        The risk-free interest rate.
    sigma :
        An initial guess for the volatility.
    payoff :
        The type of option, either 'call' or 'put'.

    Returns
    -------
        The computed implied volatility of the option. Returns NaN if the solver does not converge.
    """
    
    # Define the objective function to find the implied volatility
    obj = lambda sigma: np.abs(black_scholes(S0, K, T, r, sigma, payoff) - market_price)

    # Use fsolve to find the root of the objective function
    iv, _, solved, _ = fsolve(obj, sigma0, xtol=1e-4, maxfev=2000, full_output=True)
    return iv[0] if solved else np.nan


In [81]:
def monte_carlo_option_price(S0: float, mu: float, sigma: float, K: float, T: float, r:float, payoff: str, n_steps: int, n_simulations: int = 10_000) -> float:
    """
    Estimate the price of an option using the Monte Carlo simulation method.

    Parameters
    ----------
    S0 : float
        The current price of the underlying asset (stock price).
    mu : float
        The drift of the underlying asset.
    sigma : float
        The volatility of the underlying asset.
    K : float
        The strike price of the option.
    T : float
        The time to maturity of the option, expressed in years.
    r : float
        The risk-free interest rate.
    payoff : str
        The type of option, either 'Call' or 'Put'.
    n_steps : int
        The number of steps.
    n_simulations : int
        The number of simulated paths.

    Returns
    -------
    float
        The estimated price of the option.
    """
    # Number of time steps for each simulation
    n_steps = 100
    
    # Simulate the asset price paths using GBM
    ST = geometric_brownian_motion(S0, mu, sigma, T, n_steps, n_simulations)[:, -1]

    # Compute the payoff of the option at maturity
    if payoff.lower() == "call":
        return np.exp(-r * T) * np.mean(np.maximum(ST - K, 0))
    elif payoff.lower() == "put":
        return np.exp(-r * T) * np.mean(np.maximum(K - ST, 0))
    else:
        raise ValueError("Invalid payoff type. Choose 'call' or 'put'.")

In [93]:
def get_options_data(ticker: yf.Ticker) -> pd.DataFrame:
    expirations_date = ticker.options
    today_datetime = dt.datetime.today()

    options_data = []
    for expiration in expirations_date:
        option = ticker.option_chain(expiration)

        expiration_date = pd.to_datetime(expiration)
        days_to_maturity = (expiration_date - today_datetime).days + 1

        calls = option.calls
        puts = option.puts

        for opt, opt_type in zip([calls, puts], ["Call", "Put"]):
            opt["Days To Maturity"] = days_to_maturity
            opt["Market Price"] = (opt["ask"] + opt["bid"]) / 2
            opt["Type"] = opt_type
            options_data.append(opt)

    # Concatenate the data
    options_data = pd.concat(options_data, ignore_index=True)

    # Keep relevant columns
    options_data = options_data.rename(columns={"strike": "Strike"})
    options_data = options_data[["Days To Maturity", "Strike", "Market Price", "Type"]]

    # Remove relevant datas
    options_data = options_data[(S0 * 0.8 < options_data["Strike"]) & (options_data["Strike"] < S0 * 1.2)]
    options_data = options_data[options_data["Days To Maturity"] <= 300]

    # Set index and sort
    options_data = options_data.set_index(["Days To Maturity", "Strike", "Type"]).sort_index()

    # Pivot the table to have multi-index columns with Call/Put as the top level and Market Price as sub-levels
    options_data = options_data.pivot_table(
        index=["Days To Maturity", "Strike"],
        columns="Type",
        values=["Market Price"])

    # Reorder the multi-level columns
    options_data = options_data.reorder_levels([1, 0], axis=1).sort_index(axis=1)
    return options_data

In [94]:
options_data = get_options_data(ticker)
options_data.head()

Type                            Call          Put
                        Market Price Market Price
Days To Maturity Strike                          
4                175.0        42.050        0.015
                 180.0        36.925        0.025
                 185.0        32.000        0.035
                 190.0        26.800        0.045
                 195.0        21.800        0.075

In [96]:
# Current stock price
S0 = closes.iat[-1]

K = options_data.index.get_level_values(1)
T = options_data.index.get_level_values(0) / 365.0

# Assume 5% risk-free rate
r = 0.05

for option_type in ["Call", "Put"]:
    options_data[(option_type, "BS Price")] = black_scholes(S0, K, T, r, sigma_hat, option_type)

    ivs = np.zeros(len(options_data))
    mcs = np.zeros(len(options_data))
    for index, ((days_to_maturity, strike), row) in enumerate(options_data.iterrows()):
        maturity = days_to_maturity / 365.0
        market_price = row[(option_type, "Market Price")]
        ivs[index] = compute_implied_volatility(market_price, S0, strike, maturity, r, sigma_hat, option_type)

        # May take some time to compute, comment next line to by pass monte carlo simulation
        mcs[index] = monte_carlo_option_price(S0, mu_hat, sigma_hat, strike, maturity, r, option_type, days_to_maturity)

    options_data[(option_type, "Implied Volatility")] = ivs
    options_data[(option_type, "MC Price")] = mcs

# Reorder columns for better readability
options_data = options_data.sort_index(axis=1)
options_data.head()

Type                          Call                                             \
                          BS Price Implied Volatility   MC Price Market Price   
Days To Maturity Strike                                                         
4                175.0   40.325860           1.611894  40.966068       42.050   
                 180.0   35.328599           1.233390  35.823085       36.925   
                 185.0   30.331341           1.269787  31.034878       32.000   
                 190.0   25.334202           1.062900  25.949600       26.800   
                 195.0   20.339459           0.902793  20.979038       21.800   

Type                              Put                               \
                             BS Price Implied Volatility  MC Price   
Days To Maturity Strike                                              
4                175.0   1.893226e-10           0.412127  0.000000   
                 180.0   3.583510e-08           0.628146  0.000000   
                 185.0   3.048747e-06           0.606005  0.000000   
                 190.0   1.250426e-04           0.527677  0.000008   
                 195.0   2.643615e-03           0.463887  0.002470   

Type                                  
                        Market Price  
Days To Maturity Strike               
4                175.0         0.015  
                 180.0         0.025  
                 185.0         0.035  
                 190.0         0.045  
                 195.0         0.075

In [97]:
fig = go.Figure()
for index, option_type in enumerate(["Call", "Put"]):
    for prices_type in ["Market Price", "BS Price", "MC Price"]:
        prices = options_data.loc[options_data.index.levels[0][1], option_type][prices_type]
        fig.add_trace(go.Scatter(
            x=prices.index,
            y=prices.values,
            name=f"{option_type} {prices_type}"
        ))

fig.add_vline(x=S0, line_dash="dash", line_color="red", annotation_text="S_0")
fig.update_layout(
    title="Options Prices: Monte Carlo vs Black-Scholes vs Market",
    xaxis_title="Strike",
    yaxis_title="Price",
    height=800
)
fig.show()

#### Volatility Smile Plot

In [98]:
sample_expirations = [options_data.index.levels[0][0], options_data.index.levels[0][2],
                      options_data.index.levels[0][4], options_data.index.levels[0][6]]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Call Options', 'Put Options'))

# Plot calls
for expiry_date in sample_expirations:
    subset = options_data.loc[expiry_date, "Call"]["Implied Volatility"]

    fig.add_trace(go.Scatter(
        x=subset.index,
        y=subset.values,
        mode="markers+lines",
        name=f"Call - {expiry_date} days",
        line=dict(width=2),
        marker=dict(size=8)
    ), row=1, col=1)

# Plot puts
for expiry_date in sample_expirations:
    subset = options_data.loc[expiry_date, "Put"]["Implied Volatility"]

    fig.add_trace(go.Scatter(
        x=subset.index,
        y=subset.values,
        mode="markers+lines",
        name=f"Put - {expiry_date} days",
        line=dict(width=2),
        marker=dict(size=8)
    ), row=1, col=2)

fig.update_layout(
    title="Implied Volatility Smile",
    xaxis_title="Strike",
    yaxis_title="Implied Volatility",
    xaxis2_title="Strike",
    yaxis2_title="Implied Volatility",
    height=600,
)
fig.show()

#### Volatility Surface Plot

In [99]:
# Create the surface plot
fig = make_subplots(rows=1, cols=2, subplot_titles=("Call", "Put"),
                    specs=[[{"type": "surface"}, {"type": "surface"}]])
for index, opt_type in enumerate(["Call", "Put"]):
    plot_data = options_data[opt_type]["Implied Volatility"].unstack(0).interpolate(method="linear")

    x, y = np.meshgrid(plot_data.index, plot_data.columns)
    z = plot_data.values.T

    fig.add_trace(go.Surface(z=z, x=x, y=y, colorscale="Magma"), row=1, col=index + 1)

fig.update_layout(
    title="Implied Volatility Surface for Calls and Puts",
    scene=dict(
        xaxis_title="Strike Price",
        yaxis_title="Days To Maturity",
        zaxis_title="Implied Volatility"
    ),
    scene2=dict(
        xaxis_title="Strike Price",
        yaxis_title="Days To Maturity",
        zaxis_title="Implied Volatility"
    ),
    height=600
)
fig.show()

#### Interpretation of Implied Volatility Smile

The implied volatility smile reveals that volatility, as derived from market option prices, varies across strike prices, contradicting the Black-Scholes model’s assumption of constant volatility. The “smile” shape suggests that market participants expect higher volatility for deep out-of-the-money and in-the-money options, indicating that they anticipate more extreme price movements, along with skewness and kurtosis in asset returns. This implies fatter tails and a higher likelihood of extreme moves than a normal distribution would predict.

In a world where volatility is normally distributed, implied volatility would remain flat across strike prices. The smile, however, demonstrates that real market volatility is not constant. This highlights the limitations of the Black-Scholes model, which assumes lognormal returns and stable volatility. To better capture these dynamics, more sophisticated models like stochastic volatility (e.g., Heston) or jump-diffusion (e.g., Merton) allow for fluctuating volatility and sudden price jumps, better reflecting actual market behavior.

## 6. Stock returns are normally distributed ?

#### Histogram of log returns

In [88]:
fig = log_returns.plot(kind="hist")
fig.show()

#### Quantile-Quantile Plots

Ploting theoretical quantiles against the actual quantiles of the variable.

In [89]:
from scipy.stats import probplot

qq = probplot(log_returns, dist="norm", sparams=(1))
x = np.array([qq[0][0][0], qq[0][0][-1]])

fig = go.Figure()
fig.add_trace(go.Scatter(x=qq[0][0], y=qq[0][1], mode="markers"))
fig.add_trace(go.Scatter(x=x, y=qq[1][1] + qq[1][0] * x, mode="lines"))
fig.update_layout(
    title="Probability Plot",
    xaxis_title="Theoretical quantiles",
    yaxis_title="Ordered Values",
    width=800,
    height=600,
    showlegend=False
)
fig.show()

#### Shiparo Wilk test

The **Shapiro-Wilk** test is a statistical test used to assess whether a dataset follows a normal distribution. It is widely used in statistics to check the *normality* of data, which is a key assumption in many statistical methods and models.

- $H_0$: The data is normally distributed.
- $H_1$: The data is not normally distributed.

The test calculates a statistic based on the comparison between the ordered sample values and corresponding expected normal values. If the data is close to a normal distribution, the Shapiro-Wilk statistic will be close to 1. The test also provides a p-value to determine whether to reject the null hypothesis of normality.
- If p-value > $\alpha$: Fail to reject $H_0$, the data is likely normal.
- If p-value < $\alpha$: Reject $H_0$, the data is not normally distributed.

In [90]:
from scipy.stats import shapiro

stat, p_value = shapiro(log_returns)
print("Probably normally distributed" if p_value > 0.05 else "Probably not normally distributed")

Probably not normally distributed
